In [1]:
print("welcome")

welcome


In [2]:
%pwd

'c:\\FitnessAndWellnessChatBot\\FitnessAndWellnessChatBot\\research'

In [5]:
import os
os.chdir("../")

In [6]:
%pwd

'c:\\FitnessAndWellnessChatBot\\FitnessAndWellnessChatBot'

In [8]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents

In [10]:
extracted_data = load_pdf_files("data")

In [16]:
extracted_data

[Document(metadata={'producer': 'Prince 9.0 rev 5 (www.princexml.com)', 'creator': 'Appligent AppendPDF Pro 5.5', 'creationdate': '2019-01-30T05:47:33-08:00', 'appligent': 'AppendPDF Pro 5.5 Linux Kernel 2.6 64bit Oct  2 2014 Library 10.1.0', 'author': 'Flynn, Jellum, Howard, Moser', 'moddate': '2019-02-26T10:48:34-05:00', 'title': 'ALG: Concepts of Fitness and Wellness', 'source': 'data\\ALG Concepts of Fitness and Wellness.pdf', 'total_pages': 142, 'page': 0, 'page_label': 'i'}, page_content='GALILEO, University System of Georgia\nGALILEO Open Learning Materials\nNursing and Health Sciences Open Textbooks Nursing and Health Sciences\nSpring 2018\nConcepts of Fitness and Wellness\nScott Flynn\nGeorgia Highlands College, sflynn@highlands.edu\nLisa Jellum\nGeorgia Highlands College, ljellum@highlands.edu\nJonathan Howard\nGeorgia Highlands College, jhoward@highlands.edu\nAlthea Moser\nGeorgia Highlands College, amoser@highlands.edu\nDavid Mathis\nGeorgia Highlands College, dmathis@highl

In [13]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs


In [14]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [15]:
minimal_docs

[Document(metadata={'source': 'data\\ALG Concepts of Fitness and Wellness.pdf'}, page_content='GALILEO, University System of Georgia\nGALILEO Open Learning Materials\nNursing and Health Sciences Open Textbooks Nursing and Health Sciences\nSpring 2018\nConcepts of Fitness and Wellness\nScott Flynn\nGeorgia Highlands College, sflynn@highlands.edu\nLisa Jellum\nGeorgia Highlands College, ljellum@highlands.edu\nJonathan Howard\nGeorgia Highlands College, jhoward@highlands.edu\nAlthea Moser\nGeorgia Highlands College, amoser@highlands.edu\nDavid Mathis\nGeorgia Highlands College, dmathis@highlands.edu\nSee next page for additional authors\nFollow this and additional works at: https://oer.galileo.usg.edu/health-textbooks\nPart of the Health and Physical Education Commons\nThis Open Textbook is brought to you for free and open access by the Nursing and Health Sciences at GALILEO Open Learning Materials. It has been\naccepted for inclusion in Nursing and Health Sciences Open Textbooks by an au

In [18]:
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
        length_function=len
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [19]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 779


In [22]:
from langchain.embeddings import HuggingFaceEmbeddings
import torch

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
    )
    return embeddings
embedding = download_embeddings()

C:\Users\kumar\AppData\Local\Temp\ipykernel_61364\360052633.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\kumar\anaconda4\envs\FitnessAndWellnessChatBot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\kumar\anaconda4\envs\FitnessAndWellnessChatBot\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machin

In [23]:
vector = embedding.embed_query("Hello World")
print(len(vector))

384


In [59]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get Pinecone API key from environment
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [26]:
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY
pc = Pinecone(api_key=pinecone_api_key)

In [27]:
pc

In [32]:
from pinecone import ServerlessSpec
index_name = "fitnessdndwellnesschatbot"# index name should be in small letters.
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,  # Dimension of the embeddings
        metric="cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

index = pc.Index(index_name)

In [33]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)


In [34]:
from langchain_pinecone import PineconeVectorStore

# Connect to an existing Pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)


In [35]:
from langchain.schema import Document

# Create a new document
dswith = Document(
    page_content="dswithbappy is a youtube channel that provides tutorials on various topics.",
    metadata={"source": "Youtube"}
)

# Add the document to the Pinecone index
docsearch.add_documents(documents=[dswith])


['187da12d-3393-46fd-85ed-56d2e84ab5ea']

In [36]:
retriever = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)


In [37]:
retrieved_docs = retriever.invoke("Explain About Body Mass Index")
retrieved_docs


[Document(id='6a86a27a-300b-4f63-9d3b-4493f7bbe4a0', metadata={'source': 'data\\ALG Concepts of Fitness and Wellness.pdf'}, page_content='heart disease, diabetes, and hypertension. \n10 \n \n \nBody Mass Index \n \nIn addition to body composition and \nwaist/hip circumferences, measuring body \nmass has also been used as an effective \nmethod to assess health risks. Body mass \nindex (BMI) is a measurement of'),
 Document(id='fef7556e-a233-4811-a90d-6c230988a82f', metadata={'source': 'data\\ALG Concepts of Fitness and Wellness.pdf'}, page_content='6 \nFlynn | BODY COMPOSITION \ndensity = mass/volume. The mass \nand volume components are \nmeasured by using dry weight and \nthen weight while being submerged \nin a water tank. Since fat is less \ndense than muscle tissue, a person \nwith more body fat will weigh less in \nthe water than a similar person with \nmore lean mass. Using the \nmeasurements, the density can be \ndetermined and converted into body \nfat percentage. With a small 

In [60]:
from langchain_openai import ChatOpenAI
chatModel = ChatOpenAI(model="gpt-4o")

In [61]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [62]:
system_prompt = (
    "You are an Medical assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [63]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [66]:
response = rag_chain.invoke({"input": "who is the author of Weight	Management section in the book?"})
print(response["answer"])

The author of the "Weight Management" section is Lisa Jellum.
